# DeepRAG Quickstart

This notebook provides a minimal, runnable example of the core components of DeepRAG: the DSI and the Agent.

## 1. Setup

In [1]:
# Make sure you have installed the necessary dependencies
# !pip install -e ".[dev,viz]"


In [2]:
from hydra import compose, initialize

from deprag.configs.config import register_configs
from deprag.models.dsi import DSI
from deprag.models.agent import DeepRAGAgent
from deprag.data.docstore import DocumentStore

## 2. Load Configuration

We use Hydra to manage our configurations. We can load the default config and override parts of it easily.

In [3]:
register_configs()
with initialize(config_path="../deprag/configs", version_base="1.3"):
    cfg = compose(config_name="defaults", overrides=["data=synthetic", "model=dsi_small", "device=cpu"])

## 3. Initialize Components

### Document Store
The `DocumentStore` holds the corpus of documents that the DSI will learn to retrieve from.

In [4]:
# First, make sure the synthetic docstore exists
# !python cli.py prepare-data --config-name=data/synthetic

doc_store = DocumentStore(cfg.data.doc_store_path)
print(f"Loaded document store with {len(doc_store)} documents.")
print("Sample document ID:", doc_store.doc_ids[0])
print("Sample document text:", doc_store.get_document(doc_store.doc_ids[0]))

### DSI Model
The DSI model learns to map queries to document IDs.

In [5]:
dsi_model = DSI(cfg.model)
print("DSI model loaded (T5-small).")

### Agent Model
The Agent model learns to generate answers and decide when to retrieve.

In [6]:
# We need to switch to the agent config
with initialize(config_path="../deprag/configs", version_base="1.3"):
    agent_cfg = compose(config_name="defaults", overrides=["model=agent_small", "device=cpu"])

agent_model = DeepRAGAgent(agent_cfg.model)
print("Agent model loaded (T5-small with value head).")

## 4. Basic Inference

Let's see what the untrained models do.

### DSI Retrieval
The DSI will generate document IDs. Since it's not trained, they will be nonsensical.

In [7]:
query = "What is the capital of Canada?"
retrieved_docs = dsi_model.retrieve([query])

print(f"Query: {query}")
print(f"Top-{cfg.model.top_k} retrieved doc IDs (untrained):")
for i, doc_id in enumerate(retrieved_docs[0]):
    print(f"{i+1}: {doc_id}")

### Agent Generation
The agent will generate an answer. It also won't be very good yet.

In [8]:
input_ids = dsi_model.tokenizer(query, return_tensors="pt").input_ids

# Generate a response
# Note: We are using the DSI tokenizer here just for convenience.
# A production setup would use the agent's tokenizer.
outputs = agent_model.model.generate(input_ids, max_length=20)
response = dsi_model.tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Query: {query}")
print(f"Generated response (untrained): {response}")

## 5. Next Steps

This notebook showed how to load the core components. To make them useful, you need to train them.

- **To train the DSI**, run `python cli.py train-dsi`.
- **To train the Agent**, run `python cli.py train-agent`.

See the `02_training_walkthrough.ipynb` for more details.